In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2 as cv
import glob
import sys
import os
import copy
import argparse
import random
import tqdm
import time
from PIL import Image
import shutil
import subprocess
from multiprocessing import Pool

import torch
from torch.autograd import Variable


np.set_printoptions(suppress=True)
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 8]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Import Crop GAN related libs
gan_dir = os.path.abspath("../src/")
sys.path.append(gan_dir)
from options.train_options import TrainOptions
# from options.test_options import TestOptions

from data import create_dataset
from models import create_model
import util.util_yolo as util_yolo
from models.yolo_model import Darknet
import util.util as utils
from util.dataset_yolo import ListDataset


In [11]:
arguments = "--model double_task_cycle_gan\
             --dataroot /Users/holly/Desktop/CropGANData/model/ \
             --name Sythetic2bordenNight\
             --dataset_mode yolo_task_reverse\
             --no_flip\
             --num_threads 0\
             --gpu_ids -1\
             --display_id -1\
             --preprocess resize_and_crop\
             --load_size 256\
             --crop_size 256\
             --batch_size 1 \
             --checkpoints_dir /Users/holly/Desktop/CropGANData/model/ \
             --task_model_def ../src/config/yolov3-tiny.cfg" 

opt = TrainOptions().parse_notebook(arguments.split())

model_name:  double_task_cycle_gan
parse_notebook model_name:  double_task_cycle_gan
----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: /Users/holly/Desktop/CropGANData/model/	[default: ./checkpoints]
           continue_train: False                         
                crop_size: 256                           
          cycle_gan_epoch: 10                            
                 dataroot: /Users/holly/Desktop/CropGANData/model/	[default: None]
             dataset_mode: yolo_task_reverse             	[default: unaligned]
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            	[default: 1]
            display_ncols: 4                             
             display_port:

In [12]:
model = create_model(opt)      # create a model given opt.model and other options
model.setup(opt)               # regular setup: load and print networks; create schedulers
model.eval()


Init Resnet ... 
input_nc:  3
output_nc:  3
ngf:  64
n_blocks:  9
initialize network with normal
Init Resnet ... 
input_nc:  3
output_nc:  3
ngf:  64
n_blocks:  9
initialize network with normal
initialize network with normal
initialize network with normal

Initializing YOLO network ... 
YOLO Device:  cpu
No yolo a weights loaded 
No yolo b weights loaded 
model [DoubleTaskCycleGanModel] was created
---------- Networks initialized -------------
[Network G_A] Total number of parameters : 11.378 M
[Network G_B] Total number of parameters : 11.378 M
[Network D_A] Total number of parameters : 2.765 M
[Network D_B] Total number of parameters : 2.765 M
[Network YoloA] Total number of parameters : 8.852 M
[Network YoloB] Total number of parameters : 8.852 M
-----------------------------------------------


In [14]:
load_suffix = "/Users/holly/Desktop/CropGANData/model/Sythetic2bordenNight/latest"
model.load_networks_from_folder(load_suffix)

loading the model from /Users/holly/Desktop/CropGANData/model/Sythetic2bordenNight/latest_net_G_A.pth
loading the model from /Users/holly/Desktop/CropGANData/model/Sythetic2bordenNight/latest_net_G_B.pth
